In [53]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from utils import create_driver

from google_scholar import scrape_publications, search_from_google_scholar_profile


import time, random

True

In [7]:
with open('./data/individual_profiles.csv', 'r') as f:
    ind_profile = json.load(f)
    
other_websites = {}

for profile in ind_profile:
    if profile['google_scholar'] is None:
        other_websites[profile['full_name']] = {'other_websites': profile['other_websites']}
len(other_websites)

54

In [24]:
with open('./data/individual_profiles.json', 'r') as f:
    ind_profile = json.load(f)
    
other_websites = {}
names = []
for profile in ind_profile:
    if profile['google_scholar'] is None:
        names.append(profile['full_name'])
len(names)

54

In [25]:
# Search for profile using google scholar profile search

profile_search_url = 'https://scholar.google.com/citations?view_op=search_authors&hl=en'

driver = create_driver(debug=True)

driver.get(profile_search_url)
time.sleep(random.uniform(2,4))

results = {}
for name in names:
    results[name] = search_from_google_scholar_profile(f'{name} ntu', driver)
    time.sleep(random.uniform(1,3))
    if random.uniform(0,1)>0.75:
        time.sleep(random.uniform(5,7))

    if random.uniform(0,1)>0.8:
        driver.quit()
        driver = create_driver(debug=False)
        driver.get(profile_search_url)
        time.sleep(random.uniform(2,4))

driver.quit()

In [50]:
for k in results.keys():
    values = results[k]
    dr_ntu = next(filter(lambda s: k in s['full_name'], ind_profile))
    pubs = []

    for type in ['articles', 'books','book_chapters', 'conferences']:
        pubs += dr_ntu[type]
    values = {'possible_google_scholar_url': values, 'dr_ntu_publications': pubs}
    results[k] = values

In [55]:
possible_google_scholar = {}
for k, v in results.items():
    links = v['possible_google_scholar_url']
    possible_google_scholar.update({url:{'full_name': k} for url in links})

In [58]:
driver = create_driver(debug=True)
for url in possible_google_scholar.keys():
    driver.get(url)
    time.sleep(random.uniform(2,5))

    g_publication = scrape_publications(driver)

    possible_google_scholar[url]['google_publications'] = g_publication

    if random.uniform(0,1)>0.8:
        print('Sleeping')
        time.sleep(60)
    
    if random.uniform(0,1)>0.9:
        driver.quit()
        driver = create_driver(debug=False)

driver.quit()

Sleeping


/Users/shaojieee/.pyenv/versions/3.11.3/envs/scse_dashboard/lib/python3.11/site-packages/bs4/__init__.py:749: RuntimeWarning: coroutine 'AsyncChromiumLoader.ascrape_playwright' was never awaited
  tag = self.element_classes.get(Tag, Tag)(


Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping
Sleeping


In [105]:
for g_scholar_link, values in possible_google_scholar.items():
    name = values['full_name']
    g_pubs = values['google_publications']

    dr_ntu_pubs = results[name]['dr_ntu_publications']
    if len(dr_ntu_pubs)==0:
        continue
    cnt = 0
    for title in dr_ntu_pubs:
        for g_pub in g_pubs:
            if g_pub['title'].lower() in title.lower():
                cnt +=1
                break
    if cnt/len(dr_ntu_pubs)>0:
        if 'google_scholar' not in results[name]:
            results[name]['google_scholar'] = {'link':g_scholar_link, 'score': cnt/len(dr_ntu_pubs)}
        elif results[name]['google_scholar']['score']<cnt/len(dr_ntu_pubs):
            results[name]['google_scholar'] = {'link':g_scholar_link, 'score': cnt/len(dr_ntu_pubs)}
        

In [111]:
google_scholar = {name:v['google_scholar']['link'] for name, v in results.items() if 'google_scholar' in v}

In [112]:
len(google_scholar)

34

In [116]:
with open('./data/individual_profiles.json', 'r') as f:
    ind_profile = json.load(f)

for profile in ind_profile:
    if profile['full_name'] in google_scholar:
        profile['google_scholar'] = google_scholar[profile['full_name']]

cnt = 0
for profile in ind_profile:
    if profile['google_scholar'] is not None:
        cnt+=1
cnt


66

In [ ]:
with open('./data/individual_profiles.json', 'w') as f:
    json.dump(ind_profile, f)

In [106]:
cnt = 0

for k,i in results.items():
    if 'google_scholar' not in i and len(i['possible_google_scholar_url'])>0:
        print(k)
        print(i['possible_google_scholar_url'])
        print(i['dr_ntu_publications'])
        cnt+=1
cnt

Lin Guosheng
['https://scholar.google.com/citations?hl=en&user=ZudEhvcAAAAJ']
[]
Liu Ziwei
['https://scholar.google.com/citations?hl=en&user=lc45xlcAAAAJ']
[]
Mohamed M. Sabry
['https://scholar.google.com/citations?hl=en&user=eCo7XWkAAAAJ']
[]
Qian Kemao
['https://scholar.google.com/citations?hl=en&user=JCehBDoAAAAJ']
[]
Vidya Sudarshan
['https://scholar.google.com/citations?hl=en&user=jhD-lGAAAAAJ']
['Yu Na Wu, Vidya K Sudarshan, Shi Chao Zhu, Sung Joon Kim, Yin Hua Zhang. Functional interactions between complex I and nNOS in regulating cardiac mitochondrial activity in sham and hypertensive rat hearts. Pflugers Archiv – European Journal of Physiology, 2020; 472: 1743-1755.', 'Vidya K Sudarshan, U Rajendra Acharya, Oh Shu Lih, Muhammad Adam, Tan Jen Hong, Chua Kuang Chua, Chua Kok Poo, Tan Ru San. Automated diagnosis of congestive heart failure using dual tree complex wavelet transform and statistical features extracted from 2 seconds of ECG signals. Computers in Biology and Medicine,

5